In [ ]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

In [ ]:
snow_data = weather.query('datatype == "SNOW" and value > 0 and station.str.contains("US1NY")')
snow_data.head()

In [ ]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" AND value > 0 and station LIKE "%US1NY%"',
        connection
    )

snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

In [ ]:
weather[
    (weather.datatype == 'SNOW')
    & (weather.value > 0)
    & weather.station.str.contains('US1NY')
].equals(snow_data)

In [ ]:
station_info.id.describe()
weather.station.describe()


In [ ]:
weather.merge(station_info.rename(dict(id='station'), axis=1), on='station').sample(5, random_state=0)


In [ ]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')

right_join[right_join.datatype.isna()].head()

In [ ]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')],
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0)
    for kind in outer_join._merge.unique()
]).sort_index()